In [1]:
import pandas as pd
import os
import re
from utils_kg_search import get_all_observation,get_api_df,get_media_observation,get_media_obs_df
from call_chatgpt_with_zny_asyncio import get_gpt4api_df
from utils_data_format_conversion import *
from utils import preprocess_df
import requests

### 生成system prompt

In [ ]:
data = {
'id':['test-1'],
'turn_id':[1],
'source':['测试'],
'user-query':['世界的第三高峰是哪个'],
'Thought':['涉及事实问答，查询世界第三高峰的信息。'],
'API':['<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|kvs|>CATEGORY=>地理<|kve|><|kvs|>QUERY=>世界第三高峰<|kve|><|kvs|>TAG=>世界&第三高峰<|kve|><|api_end|>'],
'observation':["[['一些内容','一些内容','一些内容']]"],
'assistant':['世界的第三高峰是干城章嘉峰，位于喜马拉雅山脉，海拔高达8586米。'],
'relevant_label':['相关']
}
df = pd.DataFrame(data)

start_date = '2024-01-01'
end_date = '2024-03-31'
# 根据obs发布时间的max-date，生成用户提问时间user_prompt，用户提问时间比max-date随机大1-30
df = df.apply(lambda row: process_row_and_generate_prompt(row, start_date, end_date), axis=1)
# 拼接生成system prompt，地点、时间随机，用户日期根据process_row_and_generate_prompt，如果没有上面一行，自动随机生成一个时间，start_date = "2020-01-01" end_date = "2026-12-31"
df = df.apply(generate_sys_prompt,axis=1)

### csv格式转为jsonl格式

In [2]:
data = {
'id':['test-1'],
'turn_id':[1],
'source':['测试'],
'user-query':['世界的第三高峰是哪个'],
'Thought':['涉及事实问答，查询世界第三高峰的信息。'],
'API':['<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|kvs|>CATEGORY=>地理<|kve|><|kvs|>QUERY=>世界第三高峰<|kve|><|kvs|>TAG=>世界&第三高峰<|kve|><|api_end|>'],
'observation':["[['一些内容','一些内容','一些内容']]"],
'assistant':['世界的第三高峰是干城章嘉峰，位于喜马拉雅山脉，海拔高达8586米。'],
'relevant_label':['相关']
}
df = pd.DataFrame(data)
sft_df = convert_csv_to_sft(df.copy(),api_flag=True)
sft_df

,id,source,messages
0,test-1,测试,"[{'role': 'user', 'content': ['世界的第三高峰是哪个']}, ..."


### QAsearch

In [2]:
querys = {'user-query':['介绍下刘翔','姚明有多高','世界上最高的山峰是什么？']}
df = pd.DataFrame(querys)

# step 1. 生成API
api_df = get_api_df(df,category='QASearch') # category 具体设置 见函数说明
api_df = api_df.astype(str)

# step 2. 获取observation 
obs_df = get_all_observation(api_df,top_k=5,env='dev') # topk返回obs数量，env环境 dev 或 arch
obs_df

100%|██████████| 3/3 [00:07<00:00,  2.58s/it]


env dev


100%|██████████| 3/3 [00:02<00:00,  1.09it/s]

search_single_api error: 
世界最高的山峰 list index out of range


,user-query,Thought,API,API-NAME,API-CATEGORY,API-QUERY,API-TAG,observation
0,介绍下刘翔,"涉及人物知识, 查询“刘翔的简介”信息作为参考",<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|...,['QASearch'],['人物'],['刘翔的简介'],['刘翔&简介'],[[刘翔 - 知乎 刘翔，1983年7月13日出生于上海市普陀区，中国男子田径队110米栏运...
1,姚明有多高,涉及人物的身高信息，查询“姚明的身高”信息,<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|...,['QASearch'],['人物'],['姚明的身高是多少'],['姚明&身高'],[[姚明身高是多少? - 百度知道 发布时间：2017-06-08 姚明的NBA官方身高是7...
2,世界上最高的山峰是什么？,"涉及世界之最知识, 查询“世界最高的山峰”信息作为参考",<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|...,['QASearch'],['世界之最'],['世界最高的山峰'],['世界&最高&山峰'],[[世界十大山峰_百度百科 世界十大山峰分别为珠穆朗玛峰、乔戈里峰、干城章嘉峰、洛子峰、马卡...


### AUTOSearch

In [4]:
querys = {'user-query':['理想L9多少钱？','介绍下L8']}
df = pd.DataFrame(querys)

# step 1. 生成API
api_df = get_api_df(df,category='AUTOSearch') # category 具体设置 见函数说明
api_df = api_df.astype(str)

# step 2. 获取observation 
obs_df = get_all_observation(api_df,top_k=3,env='dev') # topk返回obs数量，env环境 dev 或 arch
obs_df

100%|██████████| 2/2 [00:04<00:00,  2.30s/it]


env dev


100%|██████████| 2/2 [00:00<00:00,  3.47it/s]

search_single_api error: 
理想L9多少钱？ list index out of range
search_single_api error: 
介绍下L8 list index out of range


,user-query,Thought,API,API-NAME,API-CATEGORY,API-QUERY,API-TAG,observation
0,理想L9多少钱？,涉及汽车问答，需要查询理想L9售价的信息。,<|api_start|><|kvs|>APINAME=>AUTOSearch<|kve|>...,['AUTOSearch'],['汽车'],['理想L9售价'],['理想L9&售价'],[[ 我目前无法提供准确的答案。]]
1,介绍下L8,涉及汽车问答，查询L8的简介信息做参考,<|api_start|><|kvs|>APINAME=>AUTOSearch<|kve|>...,['AUTOSearch'],['汽车'],['L8的简介'],['L8&简介'],[[理想L8简介 尺寸：理想L8传承理想ONE的经典中大型六座SUV的空间布局，车身尺寸为：...


### MEDIASearch

In [3]:
querys = {'user-query':['推荐最近的电影','推荐最新的歌','最近有什么好的电影么？']}
df = pd.DataFrame(querys)

# step 1. 生成API
api_df = get_api_df(df,category='MEDIASearch') # category 具体设置 见函数说明
api_df = api_df.astype(str)

# step 2. 获取observation
obs_df = get_media_obs_df(api_df, top_k=10, env='testtwo') # topk返回obs数量，env环境 testtwo 或 faq
obs_df

100%|██████████| 3/3 [00:07<00:00,  2.54s/it]


,user-query,Thought,API,API-NAME,API-CATEGORY,API-QUERY,API-TAG,slots,observation
0,推荐最近的电影,涉及影视推荐，需要生成影视推荐API完成内容推荐。,<|api_start|><|kvs|>APINAME=>MEDIASearch<|kve|...,['MEDIASearch'],[],['最近的电影'],[],"[{'APINAME': 'MEDIASearch', 'QUERY': '最近的电影', ...",[[《除暴安良》：刑警队长常勇在维护社会治安行动的过程中，发现一伙匪徒抢劫了大量现金并藏在了...
1,推荐最新的歌,涉及音乐相关问答，需要生成音乐推荐API完成内容推荐。,<|api_start|><|kvs|>APINAME=>MEDIASearch<|kve|...,['MEDIASearch'],[],['推荐最新的歌'],[],"[{'APINAME': 'MEDIASearch', 'QUERY': '推荐最新的歌',...",[[《做自己的太阳》：由张艺兴演唱，由Jang Woo Hyuk、Park Su Jong、...
2,最近有什么好的电影么？,涉及影视推荐，需要生成影视推荐API完成内容推荐。,<|api_start|><|kvs|>APINAME=>MEDIASearch<|kve|...,['MEDIASearch'],[],['最近有什么好的电影么？'],[],"[{'APINAME': 'MEDIASearch', 'QUERY': '最近有什么好的电...",[[《狂蟒之灾》：连年亏损的民间马戏团被团长连人带物贱卖给奸商，其中宫兰兰和阿丽以及其他团员...


In [ ]:
def get_prompts_df(df: pd.DataFrame, oneshot_prompt: str) -> pd.DataFrame:
    df['prompts'] = df.apply(lambda row: oneshot_prompt + f"""
    ---
    Background:
    ```{row['observation']}```
    Answer：
    """, axis=1)
    
    return df

In [ ]:
# step 3. 调用gpt
oneshot_prompt = '根据参考资料回答问题' # 这部分prompt可以根据自己的需求自定义
prompt_df = get_prompts_df(obs_df,oneshot_prompt)
gpt4_df = get_gpt4api_df(prompt_df, message_type='', max_request_times=1, 
                         qps=2, max_concurrent=5, asyncio_flag=False) # 各参数见函数说明

# step 4. 转成jsonl文件
gpt4_df = gpt4_df.astype(str)
gpt4_df['id'] = gpt4_df.index.astype(str)
gpt4_df['source'] = '测试'
sft_df = convert_csv_to_sft(gpt4_df,api_flag=True)
sft_df.to_json('out.jsonl', orient='records', lines=True, force_ascii=False)